In [5]:

import json
import openai 
import tiktoken


# JSON 파일 로드
with open('new_법령이름.json', 'r', encoding='utf-8') as file:
    Total_laws = json.load(file)
with open('법률메카30개_result.json', 'r', encoding='utf-8') as file:
    search_list = json.load(file)

print(len(search_list))

# 각 법의 정보를 출력

# openai.api_key = api key를 여기다 입력. 
client = openai


30


In [18]:
Total_data = []

for search in search_list:
    question = search["질문"]
    answer = search["대답"]
    answer_law = search["법령"]

    bm25 = search["bm25"]
    ann = search["ann"]
    reranked = search["ranked"]

    # BM25 ------------------------------------------------------
    bm25_use = bm25[:6]
    ann_use = ann[:6]
    ranked_use = reranked[:6]

    context_list = []

    context = ""
    i = 1
    for find in bm25_use:
        for find_law in Total_laws:
            if find_law["법이름"] == find:
                law_content = find_law["법내용"]
                context += f"{i}. 법령: {find}, 법내용: {law_content}\n"       
                i += 1 
    
    context_list.append(context)

    context = ""
    i = 1
    for find in ann_use:
        for find_law in Total_laws:
            if find_law["법이름"] == find:
                law_content = find_law["법내용"]
                context += f"{i}. 법령: {find}, 법내용: {law_content}\n"       
                i += 1 
    
    context_list.append(context)

    context = ""
    i = 1
    for find in ranked_use:
        for find_law in Total_laws:
            if find_law["법이름"] == find:
                law_content = find_law["법내용"]
                context += f"{i}. 법령: {find}, 법내용: {law_content}\n"       
                i += 1 
    
    context_list.append(context)

    encoding = tiktoken.encoding_for_model('gpt-3.5-turbo') 
    answer_size = len(encoding.encode(answer))
    text_size = int(answer_size * 0.8)



    check = 1
    for context in context_list:
        prompt_format = f"이제부터 내가 20개의 법령과 질문을 줄테니 해당 내용을 가지고 질문에 대한 답변을 해줘.\n\n{context}\n\n 이제 이 법령들을 보고 {text_size}자에 맞게 채워서 답을 해줘 질문:{question}\n\n답: " 
        prompt_format.format(context= context, quesiton= question, text_size =  text_size)
        # print(prompt_format)
        prompt = prompt_format

        message = [
            {"role": "user", "content": prompt}
        ]
        
        
        
        token_size = len(encoding.encode(prompt))
        


        if(token_size > 16000):
            
            data = {
                "질문": question,
                "검색타입" : check,
                "예측": "Token_issue",
                "토큰 수" : token_size
            }
        else:
            request_data = {
                "messages": message,
                "max_tokens": answer_size+100,
                "temperature": 0,
                "top_p": 1,
                "n": 1,
                "stream": False,
            }
            
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                **request_data,
            )
            predict=response.choices[0].message.content
            data = {
                "질문": question,
                "검색타입" : check,
                "예측": predict,
                "토큰 수" : token_size,
                "맥스 토큰 수" : answer_size
            }

        Total_data.append(data)
        check += 1




In [19]:
with open('GPT_output_6.json', 'w', encoding='utf-8') as file:
    json.dump(Total_data, file, ensure_ascii=False, indent=4)

In [4]:
prompt

'이제부터 내가 20개의 법령과 질문을 줄테니 해당 내용을 가지고 질문에 대한 답변을 해줘.\n\n1. 법령: 근로기준법제23조, 법내용: ① 사용자는 근로자에게 정당한 이유 없이 해고, 휴직, 정직, 전직, 감봉, 그 밖의 징벌(懲罰)(이하 “부당해고등”이라 한다)을 하지 못한다. ② 사용자는 근로자가 업무상 부상 또는 질병의 요양을 위하여 휴업한 기간과 그 후 30일 동안 또는 산전(産前)ㆍ산후(産後)의 여성이 이 법에 따라 휴업한 기간과 그 후 30일 동안은 해고하지 못한다. 다만, 사용자가 제84조에 따라 일시보상을 하였을 경우 또는 사업을 계속할 수 없게 된 경우에는 그러하지 아니하다.\n2. 법령: 근로기준법제24조, 법내용: ① 사용자가 경영상 이유에 의하여 근로자를 해고하려면 긴박한 경영상의 필요가 있어야 한다. 이 경우 경영 악화를 방지하기 위한 사업의 양도ㆍ인수ㆍ합병은 긴박한 경영상의 필요가 있는 것으로 본다. ② 제1항의 경우에 사용자는 해고를 피하기 위한 노력을 다하여야 하며, 합리적이고 공정한 해고의 기준을 정하고 이에 따라 그 대상자를 선정하여야 한다. 이 경우 남녀의 성을 이유로 차별하여서는 아니 된다. ③ 사용자는 제2항에 따른 해고를 피하기 위한 방법과 해고의 기준 등에 관하여 그 사업 또는 사업장에 근로자의 과반수로 조직된 노동조합이 있는 경우에는 그 노동조합(근로자의 과반수로 조직된 노동조합이 없는 경우에는 근로자의 과반수를 대표하는 자를 말한다. 이하 “근로자대표”라 한다)에 해고를 하려는 날의 50일 전까지 통보하고 성실하게 협의하여야 한다. ④ 사용자는 제1항에 따라 대통령령으로 정하는 일정한 규모 이상의 인원을 해고하려면 대통령령으로 정하는 바에 따라 고용노동부장관에게 신고하여야 한다. <개정 2010. 6. 4.> ⑤ 사용자가 제1항부터 제3항까지의 규정에 따른 요건을 갖추어 근로자를 해고한 경우에는 제23조제1항에 따른 정당한 이유가 있는 해고를 한 것으로 본다.\n3. 법령: 근로기준법제93조, 법내용: 상시 10

In [8]:
with open('GPT_output.json', 'r', encoding='utf-8') as file:
    gpt = json.load(file)

print(len(gpt))

90


In [25]:
bm25_index = [0,1,4,9,10,11,14,18,25, 26]
ann_index = [0,1,10,11,16, 18 , 25, 26]
reranked_index = [0, 6, 9, 10, 11, 13, 24 ,25, 26]


total_index = []
total_index.append(bm25_index)
total_index.append(ann_index)
total_index.append(reranked_index)
Total_data = []


encoding = tiktoken.encoding_for_model('gpt-3.5-turbo') 
answer_size = len(encoding.encode(answer))
text_size = int(answer_size * 0.8)


check = 0
idx = 0
context_list = []
for total in total_index:
    for i in total:


        question = search_list[i]["질문"]
        answer = search_list[i]["대답"]
        answer_law = search_list[i]["법령"]

        if idx==0: bm25 = search_list[i]["bm25"]
        if idx==1: ann = search_list[i]["ann"]
        if idx==2: reranked = search_list[i]["ranked"]
        
        # BM25 ------------------------------------------------------
        if idx==0: law_use = bm25[:20]
        if idx==1: law_use = ann[:20]
        if idx==2: law_use = reranked[:20]

       

        context = f"{idx}"
        context_ch = f"{idx}"
        num = 1
        for find in law_use:
            for find_law in Total_laws:
                if find_law["법이름"] == find:
                    law_content = find_law["법내용"]
                    
                    context_ch += f"{num}. 법령: {find}, 법내용: {law_content}\n" 
      
                    if len(encoding.encode(context_ch)) > 15000:
                        continue
                    else:
                        context += f"{num}. 법령: {find}, 법내용: {law_content}\n"     
                        num += 1   
                    
        context_list.append(context)
        print(check)
        for context in context_list:
            if context[0] == "0": check =1 
            if context[0] == "1": check =2 
            if context[0] == "2": check =3 

            context = context[1:]
            print(context)

            prompt_format = f"이제부터 내가 법령과 질문을 줄테니 해당 내용을 가지고 질문에 대한 답변을 해줘.\n\n{context}\n\n 이제 이 법령들을 보고 최대 400자에 맞게 채워서 질문에 대해 답을 해줘 질문:{question}\n\n답: " 
            prompt_format.format(context= context, quesiton= question)
            prompt = prompt_format

            message = [
                {"role": "user", "content": prompt}
            ]



            token_size = len(encoding.encode(prompt))



            if(token_size > 16000):
                
                data = {
                    "질문": question,
                    "검색타입" : check,
                    "예측": "Token_issue",
                    "토큰 수" : token_size
                }
            else:
                request_data = {
                    "messages": message,
                    "max_tokens": answer_size+100,
                    "temperature": 0,
                    "top_p": 1,
                    "n": 1,
                    "stream": False,
                }
                
                response = client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    **request_data,
                )
                predict=response.choices[0].message.content
                data = {
                    "질문": question,
                    "검색타입" : check,
                    "예측": predict,
                    "토큰 수" : token_size,
                    "맥스 토큰 수" : answer_size
                }

            Total_data.append(data)
        context_list = []
    idx+=1



0
1. 법령: 아동·청소년의성보호에관한법률제11조, 법내용: ① 아동ㆍ청소년성착취물을 제작ㆍ수입 또는 수출한 자는 무기 또는 5년 이상의 징역에 처한다. <개정 2020. 6. 2., 2023. 4. 11.> ② 영리를 목적으로 아동ㆍ청소년성착취물을 판매ㆍ대여ㆍ배포ㆍ제공하거나 이를 목적으로 소지ㆍ운반ㆍ광고ㆍ소개하거나 공연히 전시 또는 상영한 자는 5년 이상의 유기징역에 처한다. <개정 2020. 6. 2., 2023. 4. 11.> ③ 아동ㆍ청소년성착취물을 배포ㆍ제공하거나 이를 목적으로 광고ㆍ소개하거나 공연히 전시 또는 상영한 자는 3년 이상의 유기징역에 처한다. <개정 2020. 6. 2., 2023. 4. 11.> ④ 아동ㆍ청소년성착취물을 제작할 것이라는 정황을 알면서 아동ㆍ청소년을 아동ㆍ청소년성착취물의 제작자에게 알선한 자는 3년 이상의 유기징역에 처한다. <개정 2020. 6. 2., 2023. 4. 11.> ⑤ 아동ㆍ청소년성착취물을 구입하거나 아동ㆍ청소년성착취물임을 알면서 이를 소지ㆍ시청한 자는 1년 이상의 유기징역에 처한다. <개정 2020. 6. 2., 2023. 4. 11.> ⑥ 제1항의 미수범은 처벌한다 ⑦ 상습적으로 제1항의 죄를 범한 자는 그 죄에 대하여 정하는 형의 2분의 1까지 가중한다. <신설 2020. 6. 2.>
2. 법령: 민법제815조, 법내용: 혼인은 다음 각 호의 어느 하나의 경우에는 무효로 한다. <개정 2005. 3. 31.> 1. 당사자간에 혼인의 합의가 없는 때 2. 혼인이 제809조제1항의 규정을 위반한 때 3. 당사자간에 직계인척관계(直系姻戚關係)가 있거나 있었던 때 4. 당사자간에 양부모계의 직계혈족관계가 있었던 때 [헌법불합치, 2018헌바115, 2022.10.27, 민법(2005. 3. 31. 법률 제7427호로 개정된 것) 제815조 제2호는 헌법에 합치되지 아니한다. 위 법률조항은 2024. 12. 31.을 시한으로 개정될 때까지 계속 적용된다.]
3. 법령: 대한민국헌법제12조, 법내용: ①모든 국민

In [26]:
context_list

[]

In [27]:
with open('GPT_output2.json', 'w', encoding='utf-8') as file:
    json.dump(Total_data, file, ensure_ascii=False, indent=4)

In [6]:
total = 0

for token in gpt:
    total += token["토큰 수"]

print(int(total / len(gpt)))

6467


In [28]:
with open('GPT_output2.json', 'r', encoding='utf-8') as file:
    gpt2 = json.load(file)

print(len(gpt2))

27


In [1]:

import json
import openai 
import tiktoken

import json
with open('GPT_total.json', 'r', encoding='utf-8') as file:
    gpt3 = json.load(file)

print(len(gpt3))

# JSON 파일 로드
with open('new_법령이름.json', 'r', encoding='utf-8') as file:
    Total_laws = json.load(file)
with open('법률메카30개_result.json', 'r', encoding='utf-8') as file:
    search_list = json.load(file)

print(len(search_list))

# 각 법의 정보를 출력

# openai.api_key = api key를 여기다 입력. 
client = openai

90
30


In [23]:
import mbert_score
from rouge_score import rouge_scorer

def calculate_scores(preds, refs):
    # BERTScore
    P, R, F1 = mbert_score.score(preds, refs, lang="ko", rescale_with_baseline=True)
    
    # ROUGE-L
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    rougeL_scores = [scorer.score(ref, pred)['rougeL'] for ref, pred in zip(refs, preds)]
    
    # Aggregating ROUGE-L scores
    rougeL_precision = sum(score.precision for score in rougeL_scores) / len(rougeL_scores)
    rougeL_recall = sum(score.recall for score in rougeL_scores) / len(rougeL_scores)
    rougeL_fmeasure = sum(score.fmeasure for score in rougeL_scores) / len(rougeL_scores)

    return {
        "BERTScore": {
            "Precision": P.mean().item(),
            "Recall": R.mean().item(),
            "F1": F1.mean().item()
        },
        "ROUGE-L": {
            "Precision": rougeL_precision,
            "Recall": rougeL_recall,
            "F1": rougeL_fmeasure
        }
    }

import json
with open('GPT_output_6.json', 'r', encoding='utf-8') as file:
    gpt3 = json.load(file)

print(len(gpt3))

# JSON 파일 로드
with open('new_법령이름.json', 'r', encoding='utf-8') as file:
    Total_laws = json.load(file)
with open('법률메카30개_result.json', 'r', encoding='utf-8') as file:
    search_list = json.load(file)

print(len(search_list))


preds_bm25= []
preds_ann = []
preds_reranked = []

refs_bm25 = []
refs_ann = []
refs_reranked = []

for gp in gpt3:
    if gp["예측"] != "Token_issue":

        if gp["검색타입"] == 1:
            for ans in search_list:
                if ans["질문"] == gp["질문"]:
                    refs_bm25.append(ans["대답"])
            preds_bm25.append(gp["예측"])
        if gp["검색타입"] == 2:
            for ans in search_list:
                if ans["질문"] == gp["질문"]:
                    refs_ann.append(ans["대답"])
            preds_ann.append(gp["예측"])
        if gp["검색타입"] == 3:
                for ans in search_list:
                    if ans["질문"] == gp["질문"]:
                        refs_reranked.append(ans["대답"])
                preds_reranked.append(gp["예측"])






print(len(preds_bm25))
print(len(preds_ann))
print(len(preds_reranked))

print(len(refs_bm25))
print(len(refs_ann))
print(len(refs_reranked))         

print(preds_bm25)
print(refs_bm25)

a = calculate_scores(preds_bm25, refs_bm25)
b = calculate_scores(preds_ann, refs_ann)
c = calculate_scores(preds_reranked, refs_reranked)

90
30
30
30
30
30
30
30
['온리팬스라는 사이트에서 av배우가 아닌 2명의 남녀 일반인들이 직접 만든 성인물을 매월 구독,구매해서 보는데 그런 게시된 계정을 구독하여 이용하는 구독자들은 아동·청소년의성보호에관한법률에 따라 처벌받을 수 있습니다. 이는 아동ㆍ청소년성착취물을 구입하거나 아동ㆍ청소년성착취물임을 알면서 이를 소지ㆍ시청한 행위에 해당하기 때문입니다.\n\n상호간 촬영 배포에 동의 했다는 증거가 트위터와 카톡에 영상의 편집에 관련되거나 장난이 섞인 반응들을 간접적 증거로 보여준다면, 이는 법적으로 유효한 증거로 인정될 수 있습니다. 그러나 이는 구체적인 사건과 관련된 상황에 따라 달라질 수 있습니다.\n\n만약 위 사람들이 찍은 영상이 배포되고 시간이 지난 뒤 촬영대상자의 심경의 변화로 지금까지 찍었던 영상을 모두 내려달라한 뒤 신고를 할 경우, 구독하여 시청하던 모든 영상시청자들이 처벌받을지 여부는 해당 법률에 따라 판단될 것입니다. 이는 각각의 상황과 법률에 따라 다를 수 있습니다.', '근로기준법 제76조의3에 따르면 직장 내 괴롭힘 발생 사실을 알게 된 경우 사용자에게 신고할 수 있고, 사용자는 해당 조사를 실시하여야 합니다. 만약 소장이 괴롭힘을 가하고 있다면 해당 조사를 요청할 수 있습니다. 또한, 근로기준법 제74조에 따라 임신 중인 여성 근로자에 대한 보호조치가 있으므로 이를 참고하여 상황을 해결할 수 있을 것입니다. 또한, 민법 제1037조에 따라 상속재산의 매각이 필요한 경우 민사집행법에 따라 경매를 진행할 수 있습니다. 이러한 법령을 참고하여 상황을 해결해보시기 바랍니다.', '게임내에서 지속적인 모욕으로 인해, 본인이 상세주소와 이름을 팀채팅에서 이야기한뒤 욕하지 말아달라고 부탁을 했는데, 그후에 욕설은 멈췄지만 저의 주소와 이름을 지속적으로 전체채팅에 복사하고 붙여넣어 개인정보를 유출했는데 고소가 가능할까요?\n\n답: 개인정보보호법에 따르면 개인정보를 유출하거나 처리하는 행위는 불법이며, 고소할 수 있는 사안에 

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
print(a)
print(b)
print(c)

{'BERTScore': {'Precision': 0.6227128505706787, 'Recall': 0.5659651160240173, 'F1': 0.5899314880371094}, 'ROUGE-L': {'Precision': 0.125, 'Recall': 0.06542124542124542, 'F1': 0.06886243386243387}}
{'BERTScore': {'Precision': 0.6112195253372192, 'Recall': 0.5644535422325134, 'F1': 0.5846079587936401}, 'ROUGE-L': {'Precision': 0.08166666666666668, 'Recall': 0.058095238095238096, 'F1': 0.04738095238095239}}
{'BERTScore': {'Precision': 0.6247466802597046, 'Recall': 0.5744394659996033, 'F1': 0.5967921018600464}, 'ROUGE-L': {'Precision': 0.15277777777777776, 'Recall': 0.09115079365079365, 'F1': 0.09000721500721502}}


In [26]:
import json
with open('GPT_output_3.json', 'r', encoding='utf-8') as file:
    gpt = json.load(file)

bm_length = 0
ann_length = 0
rank_length = 0
bm_length1 = 0
ann_length1 = 0
rank_length1 = 0


encoding = tiktoken.encoding_for_model('gpt-3.5-turbo') 

for length in gpt:
    if length["검색타입"] ==1:
        bm_length += length["토큰 수"]
        if length["예측"] != "Token_issue":
            bm_length1 += len(encoding.encode(length["예측"]))
    if length["검색타입"] ==2:
        ann_length += length["토큰 수"]
        if length["예측"] != "Token_issue":
            ann_length1 += len(encoding.encode(length["예측"]))
    if length["검색타입"] ==3:
        rank_length += length["토큰 수"]
        if length["예측"] != "Token_issue":
            rank_length1 += len(encoding.encode(length["예측"]))
    
print(len(gpt))

print(bm_length/30)
print(ann_length/30)
print(rank_length/30)

90
2446.233333333333
1937.5666666666666
1574.0333333333333
